In [4]:
import pickle
from collections import defaultdict
from collections import Counter
from scipy.special import logit, expit

In [5]:
def reccomend(a1=None,a2=None,a3=None,a4=None,a5=None,
              e1=None,e2=None,e3=None,e4=None,e5=None,
              enemy_laner1=None,enemy_laner2=None):
    
    allies=[champid for champid in [a1,a2,a3,a4,a5] if champid != None]
    enemies=[champid for champid in [e1,e2,e3,e4,e5] if champid != None]
    return allies,enemies

In [6]:
result=reccomend(a1=1,a2=8,e1=133,e2=178,e3=203)
result

([1, 8], [133, 178, 203])

In [8]:
try:
    champ_dict
except:
    champ_dict = pickle.load(open("champion_dict.p","rb"))

def champion_number_to_name(champion_number):

    return champ_dict["data"][str(champion_number)]["name"]

In [9]:
possible_champs=champ_dict["data"].keys()
predicting_probs={}

In [165]:
class recommender(object):
    def __init__(self,a1=None,a2=None,a3=None,a4=None,a5=None,
                 e1=None,e2=None,e3=None,e4=None,e5=None,
                 enemy_laner1=None,enemy_laner2=None):
        #self.a1=a1, self.a2=a2, self.a3=a3, self.a4=a4, self.a5=a5
        #self.e1=e1, self.e2=e2, self.e3=e3, self.e4=e4, self.e5=e5
        self.enemy_laners=[champid for champid in [enemy_laner1,enemy_laner2] if champid != None]
        self.allies=[champid for champid in [a1,a2,a3,a4,a5] if champid != None]
        self.enemies=[champid for champid in [e1,e2,e3,e4,e5] if champid != None]
        pickle_path="/home/bryce/galvanize/class/capstone/"
        self.champion_dict = pickle.load(open(pickle_path+"picklejar/champion_dict.p","rb"))
        self.matchup_win_percent = pickle.load(open(pickle_path+"picklejar/trimmed_matchup.p","rb"))
        self.synergy_win_percent = pickle.load(open(pickle_path+"picklejar/trimmed_synergy.p","rb"))
        self.flat_win_percent = pickle.load( open(  pickle_path+"picklejar/flat_win_percent.p", "rb" ) )
        self.lane_matchup= pickle.load( open( pickle_path+"picklejar/trimmed_lane.p", "rb" ) )
        self.predicted_winrates = {}
        self.update_events=0
        
        
    def load_base_winrate(self):
        for key in self.flat_win_percent:
            self.predicted_winrates[key]=logit(.5) #self.flat_win_percent[key]

###START
    def get_winrate_for_enemy(self,possible_pick,enemyid):
        return self.matchup_win_percent[possible_pick,enemyid]
    def get_winrate_for_ally(self,possible_pick,allyid):
        return self.synergy_win_percent[possible_pick,allyid]
    def get_winrate_for_enemy_laner(self, possible_pick,lanerid):
        return self.lane_matchup[possible_pick,lanerid]
    
    
    def update_predicted_winrate_for_enemy(self,picked_champion):
        self.update_events+=1   
        for champion in self.predicted_winrates.keys():
            if champion == picked_champion:
                pass
            else:
                self.predicted_winrates[champion] = (
                    self.predicted_winrates[champion] + 
                    logit(self.get_winrate_for_enemy(champion,picked_champion)[0]))   
    
    def update_predicted_winrate_for_ally(self,picked_champion):
        self.update_events+=1
        for champion in self.predicted_winrates.keys():
            if champion == picked_champion:
                pass
            else:
                self.predicted_winrates[champion] = (
                    self.predicted_winrates[champion] + 
                    logit(self.get_winrate_for_ally(champion,picked_champion)[0]))
    
    def update_predicted_winrate_for_lane(self, picked_champion):
        self.update_events+=1
        for champion in self.predicted_winrates.keys():
            if champion == picked_champion:
                pass
            else:
                self.predicted_winrates[champion] = (
                    self.predicted_winrates[champion] + 
                    logit(self.get_winrate_for_enemy_laner(champion,picked_champion)[0]))
###END           
    
    def top_picks(self):
        return sorted(self.predicted_winrates.items(), key=lambda(item): item[1], reverse=True)
    
    def champion_name_from_id(self,champion_number):
        return self.champion_dict["data"][str(champion_number)]["name"]
    
    def predict(self,n=5):
        top_picks=[]
        for pair in self.top_picks()[:n]:
            top_picks.append((self.champion_name_from_id(pair[0]), "%.2f" % expit(pair[1]/(self.update_events+1))))
        return top_picks
        
        
        

In [164]:
for key, value in model.lane_matchup.items():
    if type(value) == float:
        print key, value

(421, 117) 0.5
(51, 121) 0.5
(121, 110) 0.5
(254, 127) 0.5
(77, 34) 0.5
(113, 112) 0.5
(42, 60) 0.5
(117, 77) 0.5
(48, 136) 0.5
(72, 30) 0.5
(77, 117) 0.5
(427, 10) 0.5
(7, 33) 0.5
(427, 82) 0.5
(14, 163) 0.5
(203, 55) 0.5
(163, 60) 0.5
(102, 161) 0.5
(72, 114) 0.5
(115, 421) 0.5
(60, 13) 0.5
(266, 163) 0.5
(56, 30) 0.5
(32, 7) 0.5
(69, 427) 0.5
(12, 2) 0.5
(60, 240) 0.5
(240, 134) 0.5
(27, 203) 0.5
(421, 115) 0.5
(41, 32) 0.5
(26, 72) 0.5
(421, 134) 0.5
(50, 421) 0.5
(134, 107) 0.5
(119, 254) 0.5
(60, 163) 0.5
(77, 134) 0.5
(82, 427) 0.5
(203, 13) 0.5
(72, 201) 0.5
(101, 120) 0.5
(69, 120) 0.5
(72, 1) 0.5
(163, 266) 0.5
(33, 1) 0.5
(127, 421) 0.5
(60, 55) 0.5
(120, 103) 0.5
(127, 12) 0.5
(69, 20) 0.5
(427, 34) 0.5
(112, 154) 0.5
(266, 267) 0.5
(121, 42) 0.5
(7, 72) 0.5
(127, 154) 0.5
(121, 85) 0.5
(127, 107) 0.5
(154, 61) 0.5
(38, 72) 0.5
(72, 90) 0.5
(161, 102) 0.5
(421, 112) 0.5
(103, 2) 0.5
(60, 101) 0.5
(112, 427) 0.5
(55, 113) 0.5
(48, 112) 0.5
(115, 56) 0.5
(15, 121) 0.5
(77, 11

In [132]:
model=recommender(a1=1,a2=8,e1=3,e2=4,e3=26)

In [160]:
model.predicted_winrates[14]

-0.62883559117992571

In [133]:
model.enemies


[3, 4, 26]

In [134]:
model.predicted_winrates

{}

In [399]:
sorted(model.synergy_win_percent.items(),reverse=True, key=lambda x: x[1])

[((20, 421), 1.0),
 ((72, 60), 1.0),
 ((103, 103), 1.0),
 ((203, 72), 1.0),
 ((120, 72), 1.0),
 ((72, 203), 1.0),
 ((421, 20), 1.0),
 ((136, 42), 1.0),
 ((72, 120), 1.0),
 ((127, 112), 1.0),
 ((72, 33), 1.0),
 ((112, 127), 1.0),
 ((39, 39), 1.0),
 ((77, 60), 1.0),
 ((48, 48), 1.0),
 ((134, 134), 1.0),
 ((33, 72), 1.0),
 ((34, 34), 1.0),
 ((127, 30), 1.0),
 ((36, 36), 1.0),
 ((60, 77), 1.0),
 ((119, 119), 1.0),
 ((18, 18), 1.0),
 ((69, 69), 1.0),
 ((60, 72), 1.0),
 ((21, 21), 1.0),
 ((30, 127), 1.0),
 ((78, 78), 1.0),
 ((42, 136), 1.0),
 ((203, 32), 0.9),
 ((32, 203), 0.9),
 ((136, 103), 0.9),
 ((103, 136), 0.9),
 ((240, 122), 0.875),
 ((203, 154), 0.875),
 ((60, 120), 0.875),
 ((122, 240), 0.875),
 ((120, 60), 0.875),
 ((154, 203), 0.875),
 ((127, 163), 0.8571428571428571),
 ((163, 127), 0.8571428571428571),
 ((33, 203), 0.8181818181818182),
 ((203, 33), 0.8181818181818182),
 ((20, 56), 0.8),
 ((40, 201), 0.8),
 ((164, 58), 0.8),
 ((58, 164), 0.8),
 ((201, 40), 0.8),
 ((154, 120), 0.8)

In [135]:
model.load_base_winrate()

In [136]:
model.predict()

[(u'Annie', '0.50'),
 (u'Olaf', '0.50'),
 (u'Galio', '0.50'),
 (u'Twisted Fate', '0.50'),
 (u'Xin Zhao', '0.50')]

In [137]:
model.enemies, model.allies

([3, 4, 26], [1, 8])

In [138]:
model.update_predicted_winrate_for_enemy(8)

In [139]:
model.predicted_winrates

{1: -0.19105523676270936,
 2: 0.090971778205726592,
 3: 0.059963464767557477,
 4: -0.1124779834266902,
 5: -0.17185025692665942,
 6: 0.020000666706669435,
 7: -0.21440987134545506,
 8: 0.0,
 9: -0.48653293119694629,
 10: -0.012270092591814247,
 11: -0.012320484388040736,
 12: 0.22024079165624816,
 13: 0.0088889474172459942,
 14: -0.26311081172127548,
 15: 0.070951735972284394,
 16: -0.22250436944521268,
 17: -0.043802622658392999,
 18: 0.015314234973042356,
 19: -0.25878760645206594,
 20: 0.23767165187711939,
 21: -0.038764543450244424,
 22: 0.066301079821240075,
 23: 0.011236073266925752,
 24: -0.010582109330537083,
 25: 0.19870952597692768,
 26: -0.10697211955216783,
 27: -0.26875406256626216,
 28: -0.0090772181511167629,
 29: -0.13976194237515885,
 30: -0.11778303565638351,
 31: -0.13187804421813529,
 32: 0.064538521137571164,
 33: 0.073687715791159333,
 34: 0.0086580627431145311,
 35: -0.031748698314580298,
 36: -0.076540077122334488,
 37: -0.23336305714769345,
 38: -0.120627987788

In [140]:
model.predict()

[(u'Tahm Kench', '0.55'),
 (u'Corki', '0.54'),
 (u'Nunu', '0.53'),
 (u'Lucian', '0.53'),
 (u'Alistar', '0.53')]

In [141]:
model.update_predicted_winrate_for_lane(4)

In [142]:
model.predict()

[(u'Kennen', '0.58'),
 (u'Evelynn', '0.56'),
 (u'Gragas', '0.55'),
 (u'Lissandra', '0.55'),
 (u'LeBlanc', '0.54')]

In [357]:
model.update_predicted_winrate(9)

In [358]:
model.predict()

Ryze 0.571786991032
Cassiopeia 0.559481860222
Corki 0.558665552711
Karthus 0.557682453201
Nunu 0.55669039197


In [359]:
model.update_predicted_winrate(57,enemy=False)

In [360]:
model.predict()

Ryze 0.572244656349
Tahm Kench 0.562200837852
Nidalee 0.555119791733
Cassiopeia 0.553612221434
Poppy 0.551810774787


In [320]:
projected = sorted(flat_wr.items(), key=lambda x: x[1], reverse=True)
for pair in projected[:5]:
    print champion_name_from_id(pair[0]),pair[1]

Ryze 0.576932111816
Nunu 0.559818625741
Evelynn 0.546513856415
LeBlanc 0.545976120329
Nidalee 0.545306001188


In [43]:
def run_and_predict(a1=None,a2=None,a3=None,a4=None,a5=None,
                 e1=None,e2=None,e3=None,e4=None,e5=None,
                 enemy_laner1=None,enemy_laner2=None):
    model=recommender(**locals())
    model.load_base_winrate()
    for champ in model.enemies:
        model.update_predicted_winrate_for_enemy(champ)
    for champ in model.allies:
        model.update_predicted_winrate_for_ally(champ)
    for champ in model.enemy_laners:
        model.update_predicted_winrate_for_lane(champ)
    return model.predict()

In [68]:
test=run_and_predict(a1=1,a2=8,e1=3,e2=4,e3=26,a3=39, enemy_laner1=7, enemy_laner2=14)
test

[(u'Evelynn', '0.55'),
 (u'Nunu', '0.54'),
 (u'LeBlanc', '0.54'),
 (u'Braum', '0.54'),
 (u'Ryze', '0.53')]

In [89]:
html=""
for line in test:
    html+=line[0]+" "*(15-len(line[0]))+line[1]+"\n"

print html


Evelynn        0.55
Nunu           0.54
LeBlanc        0.54
Braum          0.54
Ryze           0.53



In [169]:
[x for x
  in range(6)]

[0, 1, 2, 3, 4, 5]